In [1]:
import os
import pymysql
import pandas as pd

host = os.getenv('MYSQL_HOST')
port = os.getenv('MYSQL_PORT')
user = os.getenv('MYSQL_USER')
password = os.getenv('MYSQL_PASSWORD')
database = os.getenv('MYSQL_DATABASE')

conn = pymysql.connect(
    host=host,
    port=int(3306),
    user="root",
    passwd="Blazers#88",
    db="pge_schema",
    charset='utf8mb4')

In [2]:
# query = ("SELECT first_name, last_name, hire_date FROM employees "
#          "WHERE hire_date BETWEEN %s AND %s")

# hire_start = datetime.date(2006, 1, 1)
# hire_end = datetime.date(2006, 12, 31)

# cursor.execute(query, (hire_start, hire_end))


You have been tasked with evaluating a US equity portfolio whose holdings, financial characteristics, and performance is provided. All findings should come from the provided information, and be explainable at a high level to someone who isn't familiar with this data.

1.       What is the portfolio's weighted average return on equity ratio (ROE) over time?

2.       What is the portfolio's approximate quarterly performance for each quarter? Assume that cash returns are 0%.

3.       What appears to be the investment approach or style of this portfolio, and how has this contributed to or detracted from performance over time? Feel free to provide metrics or illustrations that may help.

4.       Are there any concerns with the portfolio and/or data that you believe may be worth further investigation, and what are some potential next steps to continue the analysis?



#### Checking and cleaning the data

In [3]:
df = pd.read_sql_query("SELECT * FROM financials", conn)
df.tail(10)

,Attribute,Ticker,Year,Value
71026,Free_Cash_Flow,ZTS,2017,916.000
71027,Free_Cash_Flow,ZTS,2018,1209.000
71028,Free_Cash_Flow,ZTS,2019,1021.000
71029,Free_Cash_Flow,ZTS,2020,1293.000
71030,Free_Cash_Flow,ZS,2015,-9.984
71031,Free_Cash_Flow,ZS,2016,-18.163
71032,Free_Cash_Flow,ZS,2017,-14.193
71033,Free_Cash_Flow,ZS,2018,2.137
71034,Free_Cash_Flow,ZS,2019,29.345
71035,Free_Cash_Flow,ZS,2020,35.112


In [4]:
returns = "SELECT * FROM returns"
df = pd.read_sql_query(returns, conn)
df.tail(10)

,Ticker,Qtr_Ending,Pct_Ret_Qtr,Mkt_Cap,Price
38600,ZS,9/30/18,14.069927,4228.86684,40.78
38601,ZS,12/31/18,-3.849924,4431.22674,39.21
38602,ZS,3/31/19,80.897736,5992.89789,70.93
38603,ZS,6/30/19,8.050192,8576.45712,76.64
38604,ZS,9/30/19,-38.335070,10723.61031,47.26
38605,ZS,12/31/19,-1.608121,5626.18548,46.50
38606,ZS,3/31/20,30.881727,7250.52994,60.86
38607,ZS,6/30/20,79.921130,8743.67676,109.50
38608,ZS,9/30/20,28.484022,17246.28745,140.69
38609,ZS,12/31/20,41.950394,18212.62725,199.71


In [5]:
### copy and paste questions

#### 1.       What is the portfolio's weighted average return on equity ratio (ROE) over time?

In [6]:
# holdings and financials
weighted_roe_holdings = """
WITH weight_roe AS
(
SELECT holdings.Company_Name, holdings.Ticker, holdings.Weight/100, YEAR(STR_TO_DATE(Date, '%m/%d/%Y')) AS H_Year, 
financials.Attribute, financials.Year, financials.Value,
(holdings.Weight/100) * (financials.Value) AS weighted_roe
FROM holdings
INNER JOIN financials on holdings.Ticker = financials.Ticker
)

SELECT H_Year, Attribute, AVG(weighted_roe)
FROM weight_roe
WHERE Attribute = 'Return_on_Equity'
GROUP BY H_Year, Attribute
ORDER BY H_Year
"""

weighted_roe_holdings = pd.read_sql_query(weighted_roe_holdings, conn)
weighted_roe_holdings


,H_Year,Attribute,AVG(weighted_roe)
0,2001,Return_on_Equity,-0.250219
1,2002,Return_on_Equity,-0.435867
2,2003,Return_on_Equity,-0.132067
3,2004,Return_on_Equity,0.053968
4,2005,Return_on_Equity,0.079171
5,2006,Return_on_Equity,0.150694
6,2007,Return_on_Equity,0.085082
7,2008,Return_on_Equity,0.043139
8,2009,Return_on_Equity,0.102899
9,2010,Return_on_Equity,0.162252


In [7]:
# visual

In [8]:
# holdings and return

weighted_price_holdings = """
WITH weighted_price AS
(
SELECT holdings.Company_Name, holdings.Ticker, holdings.Weight/100 AS Weight,
returns.Qtr_Ending, returns.Pct_Ret_Qtr/100 AS Quarterly_Returns, returns.Mkt_Cap, returns.Price,
Weight/100 * Price AS Weighted_Price
FROM holdings
INNER JOIN returns ON holdings.Ticker = returns.Ticker
WHERE holdings.Date = Qtr_Ending
ORDER BY Qtr_Ending
)
SELECT STR_TO_DATE(Qtr_Ending, '%m/%d/%Y') AS Qtr_Ending, SUM(Weighted_Price) AS Weighted_Price
FROM weighted_price
GROUP BY Qtr_Ending
ORDER BY Qtr_Ending
"""

weighted_price_holdings = pd.read_sql_query(weighted_price_holdings, conn)
weighted_price_holdings


,Qtr_Ending,Weighted_Price
0,2001-12-31,172.511678
1,2002-03-31,83.163469
2,2002-06-30,44.293435
3,2002-09-30,33.125195
4,2002-12-31,20.469355
...,...,...
72,2019-12-31,154.030271
73,2020-03-31,129.879971
74,2020-06-30,210.174967
75,2020-09-30,234.766103


In [9]:
weighted_price_holdings['Weighted_Price'].iloc[0]

172.5116782351817

In [10]:
weighted_price_holdings['Weighted_Price'].iloc[-1]/weighted_price_holdings['Weighted_Price'].iloc[0] - 1

0.8763658392061988

In [11]:
## cash returns



cash_returns = """
SELECT Company_Name, Ticker, STR_TO_DATE(Date, "%m/%d/%Y") AS Date, Weight/100 AS Weight
FROM holdings
WHERE Ticker LIKE "US%"
ORDER BY Date
"""

cash_returns = pd.read_sql_query(cash_returns, conn)
cash_returns


,Company_Name,Ticker,Date,Weight
0,/ -IND,US,2001-09-30,0.002123
1,US Dollar,USD_CASH,2001-09-30,0.037992
2,US Dollar,USD_CASH,2001-12-31,0.021540
3,US Dollar,USD_CASH,2002-03-31,0.038307
4,US Dollar,USD_CASH,2002-06-30,0.061789
...,...,...,...,...
82,US Dollar,USD_CASH,2020-06-30,0.080082
83,US Dollar,USD_CASH,2020-09-30,0.063812
84,US Dollar,USD_CASH,2020-12-31,0.071369
85,US Dollar,USD_CASH,2021-03-31,0.079754


#### 2.       What is the portfolio's approximate quarterly performance for each quarter? Assume that cash returns are 0%.

In [12]:
weighted_quarterly_return = """
WITH weighted_return AS
(
SELECT holdings.Company_Name, holdings.Ticker, holdings.Weight/100 AS Weight,
returns.Qtr_Ending, returns.Pct_Ret_Qtr/100 AS Quarterly_Returns, returns.Mkt_Cap, returns.Price,
Weight/100 * Pct_Ret_Qtr AS Weighted_Return
FROM holdings
INNER JOIN returns ON holdings.Ticker = returns.Ticker
WHERE holdings.Date = Qtr_Ending
ORDER BY Qtr_Ending
)
SELECT STR_TO_DATE(Qtr_Ending, '%m/%d/%Y') AS Qtr_Ending, SUM(Weighted_Return) AS Weighted_Return
FROM weighted_return
GROUP BY Qtr_Ending
ORDER BY Qtr_Ending
"""

weighted_quarterly_return = pd.read_sql_query(weighted_quarterly_return, conn)
weighted_quarterly_return

,Qtr_Ending,Weighted_Return
0,2001-12-31,46.510469
1,2002-03-31,-2.063081
2,2002-06-30,-10.545153
3,2002-09-30,-9.776038
4,2002-12-31,14.779390
...,...,...
72,2019-12-31,10.634220
73,2020-03-31,-5.254564
74,2020-06-30,46.867945
75,2020-09-30,11.062661


In [13]:
weighted_quarterly_return['Weighted_Return'].describe()

count     77.000000
mean       8.196363
std       25.126741
min      -16.886302
25%        0.502215
50%        5.207775
75%       10.634220
max      204.923871
Name: Weighted_Return, dtype: float64

In [14]:
## cash returns



cash_returns = """
SELECT Company_Name, Ticker, STR_TO_DATE(Date, "%m/%d/%Y") AS Date, Weight/100 AS Weight
FROM holdings
WHERE Ticker LIKE "US%"
ORDER BY Date
"""

cash_returns = pd.read_sql_query(cash_returns, conn)
cash_returns


,Company_Name,Ticker,Date,Weight
0,/ -IND,US,2001-09-30,0.002123
1,US Dollar,USD_CASH,2001-09-30,0.037992
2,US Dollar,USD_CASH,2001-12-31,0.021540
3,US Dollar,USD_CASH,2002-03-31,0.038307
4,US Dollar,USD_CASH,2002-06-30,0.061789
...,...,...,...,...
82,US Dollar,USD_CASH,2020-06-30,0.080082
83,US Dollar,USD_CASH,2020-09-30,0.063812
84,US Dollar,USD_CASH,2020-12-31,0.071369
85,US Dollar,USD_CASH,2021-03-31,0.079754


#### 3.       What appears to be the investment approach or style of this portfolio, and how has this contributed to or detracted from performance over time? Feel free to provide metrics or illustrations that may help.

In [15]:
## Growth:

# initial negative roe with an upward trend over time, notable jump during economic recovery following ~2010
# volatile returns - high standard deviation


#### 4.       Are there any concerns with the portfolio and/or data that you believe may be worth further investigation, and what are some potential next steps to continue the analysis?

In [16]:
## I didn't check for/clean missing values

In [17]:

# Taking the time to look at plots
#      -time series plots of rolling return and std deviations, scatter plots, histograms

# Calculating the Sharpe ratio

# Comparing performance against a suitable benchmark

# Bucketizing companies by financials and comparing distributions

# Bucketizing by returns and looking at recurring performers

# Using the historical returns, can use time-series forecasting to predict future returns or buy/sell/hold signals
